In [92]:
import pandas as pd
import numpy as np

In [93]:
df = pd.read_csv('~/Downloads/annual_conc_by_monitor_2016.csv')
df2 = pd.read_csv('~/Downloads/annual_conc_by_monitor_2017.csv')
df3 = pd.read_csv('~/Downloads/annual_conc_by_monitor_2018.csv')

In [94]:
df.append(df3)
df.append(df2)

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,...,75th Percentile,50th Percentile,10th Percentile,Local Site Name,Address,State Name,County Name,City Name,CBSA Name,Date of Last Change
0,1,3,10,44201,1,30.497478,-87.880258,NAD83,Ozone,1 HOUR,...,0.054,0.046,0.029,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2017-01-04
1,1,3,10,44201,1,30.497478,-87.880258,NAD83,Ozone,8-HR RUN AVG BEGIN HOUR,...,0.050,0.042,0.025,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2017-01-04
2,1,3,10,44201,1,30.497478,-87.880258,NAD83,Ozone,8-HR RUN AVG BEGIN HOUR,...,0.050,0.042,0.025,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2017-01-04
3,1,3,10,44201,1,30.497478,-87.880258,NAD83,Ozone,8-HR RUN AVG BEGIN HOUR,...,0.050,0.041,0.025,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2017-01-04
4,1,3,10,68101,1,30.497478,-87.880258,NAD83,Sample Flow Rate- CV,24 HOUR,...,0.200,0.000,0.000,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2016-10-05
5,1,3,10,68102,1,30.497478,-87.880258,NAD83,Sample Volume,24 HOUR,...,24.000,24.000,24.000,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2016-10-05
6,1,3,10,68105,1,30.497478,-87.880258,NAD83,Average Ambient Temperature,24 HOUR,...,25.400,20.200,8.100,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2016-10-05
7,1,3,10,68108,1,30.497478,-87.880258,NAD83,Average Ambient Pressure,24 HOUR,...,761.000,759.000,755.000,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2016-10-05
8,1,3,10,68109,1,30.497478,-87.880258,NAD83,Elapsed Sample Time,24 HOUR,...,1440.000,1440.000,1439.000,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2016-10-05
9,1,3,10,88101,1,30.497478,-87.880258,NAD83,PM2.5 - Local Conditions,24 HOUR,...,8.600,6.900,4.400,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2017-03-21


In [95]:
df = df[df['State Name']=='California']

In [96]:
df.reset_index(drop=True,inplace=True)

In [97]:
df = df[df['CBSA Name']=='San Francisco-Oakland-Hayward, CA']

In [102]:
df.drop(['State Code', 'County Code','State Name','City Name','County Name', 'Datum'], axis=1)

,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,Pollutant Standard,Metric Used,...,98th Percentile,95th Percentile,90th Percentile,75th Percentile,50th Percentile,10th Percentile,Local Site Name,Address,CBSA Name,Date of Last Change
0,7,42601,1,37.687526,-121.784217,WGS84,Nitric oxide (NO),1 HOUR,NaN,Observed Values,...,42.2000,24.9000,12.6000,2.6000,0.9000,0.2000,Livermore,793 Rincon Ave.,"San Francisco-Oakland-Hayward, CA",2017-02-16
1,7,42602,1,37.687526,-121.784217,WGS84,Nitrogen dioxide (NO2),1 HOUR,NO2 1-hour,Daily Maximum 1-hour average,...,34.4000,31.4000,28.6000,24.2000,18.0000,5.4000,Livermore,793 Rincon Ave.,"San Francisco-Oakland-Hayward, CA",2017-02-16
2,7,42602,1,37.687526,-121.784217,WGS84,Nitrogen dioxide (NO2),1 HOUR,NO2 Annual 1971,Observed values,...,26.2000,23.0000,19.6000,12.3000,6.1000,1.8000,Livermore,793 Rincon Ave.,"San Francisco-Oakland-Hayward, CA",2017-02-16
3,7,42603,1,37.687526,-121.784217,WGS84,Oxides of nitrogen (NOx),1 HOUR,NaN,Observed Values,...,63.0000,45.8000,33.1000,15.1000,7.3000,2.3000,Livermore,793 Rincon Ave.,"San Francisco-Oakland-Hayward, CA",2017-02-16
4,7,43141,6,37.687526,-121.784217,WGS84,n-Dodecane,1 HOUR,NaN,Observed Values,...,0.5019,0.4031,0.3039,0.1837,0.1122,0.0198,Livermore,793 Rincon Ave.,"San Francisco-Oakland-Hayward, CA",2018-03-05
5,7,43202,6,37.687526,-121.784217,WGS84,Ethane,1 HOUR,NaN,Observed Values,...,21.2689,15.9229,10.8884,5.9382,3.9586,2.3638,Livermore,793 Rincon Ave.,"San Francisco-Oakland-Hayward, CA",2018-03-05
6,7,43203,6,37.687526,-121.784217,WGS84,Ethylene,1 HOUR,NaN,Observed Values,...,5.2928,3.6648,2.4447,1.3184,0.7638,0.0000,Livermore,793 Rincon Ave.,"San Francisco-Oakland-Hayward, CA",2018-03-05
7,7,43204,6,37.687526,-121.784217,WGS84,Propane,1 HOUR,NaN,Observed Values,...,15.9641,11.4435,8.0007,3.4139,1.9796,0.8560,Livermore,793 Rincon Ave.,"San Francisco-Oakland-Hayward, CA",2018-03-05
8,7,43205,6,37.687526,-121.784217,WGS84,Propylene,1 HOUR,NaN,Observed Values,...,4.2545,3.0346,2.0311,1.0407,0.6703,0.3712,Livermore,793 Rincon Ave.,"San Francisco-Oakland-Hayward, CA",2018-03-05
9,7,43206,6,37.687526,-121.784217,WGS84,Acetylene,1 HOUR,NaN,Observed Values,...,6.6750,4.5290,3.0469,1.6864,1.1127,0.6184,Livermore,793 Rincon Ave.,"San Francisco-Oakland-Hayward, CA",2018-03-05
